# PyTorch Prediction
2 Layers CNN + Pool
---

Credits:
- https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/cifar-cnn/cifar10_cnn_solution.ipynb
- Balakrishnan, Anusha, and Alisha Rege. 
Reading Emotions from Speech Using Deep Neural Networks. 
Stanford, 2017, pp. 1–8, Reading Emotions from Speech Using Deep Neural Networks.


In [11]:
#Torch
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch import Tensor

import torch.nn as nn
import torch.optim as optim

# sklearn
from sklearn.model_selection import train_test_split

#Scipy
import scipy

#Numpy
import numpy as np

#Pandas
import pandas as pd

#Matplot
import matplotlib.pyplot as plt

#Librosa
import librosa
import librosa.display

# Other  
import os

# Local
from source_pytorch.newDataset import NewDatset
from source_pytorch.newModel import NewModel

In [13]:
#Set Audio CSV dataset file
full_set = pd.read_csv(os.path.join('.', "ReferenceData.csv")).loc[:196] #test filter

#Split the Train and Test dataset
X_train, X_test, y_train, y_test = train_test_split(full_set["fname"]
                                                    , full_set["label"]
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=50
                                                   )
#Split the Train and Validation dataset
X_train, X_valid , y_train, y_valid = train_test_split(X_train
                                                    , y_train
                                                    , test_size=0.25
                                                    , shuffle=True
                                                    , random_state=50
                                                   )


#Merge data and prediction in 1 dataset for train validation and test
train_df = pd.concat([X_train,y_train], axis = 1)  
valid_df =  pd.concat([X_valid,y_valid], axis = 1)
test_df = pd.concat([X_test,y_test], axis = 1)

print('Dataset Len:\t Train - {} \tValid Dataset - {} \tTest Dataset - {}'.format(len(train_df), len(valid_df), len(test_df)))

Dataset Len:	 Train - 110 	Valid Dataset - 37 	Test Dataset - 50


In [14]:
#set Batch Size
batch_size = 10

#Compose DataLoader
train_dataloader = DataLoader(
    NewDatset(dataframe=train_df), batch_size=batch_size, shuffle=True, num_workers=0)

valid_dataloader = DataLoader(
    NewDatset(dataframe=valid_df), batch_size=batch_size, shuffle=True, num_workers=0)

In [15]:
#Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Call the model
model = NewModel()
model.to(device)
print(model)

NewModel(
  (conv): Conv2d(1, 1280, kernel_size=(3, 3), stride=(1, 1))
  (batch): BatchNorm2d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4158720, out_features=8, bias=True)
  (drop): Dropout2d(p=0.2, inplace=False)
)


In [16]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [17]:
%%time
#Credits: https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/cifar-cnn/cifar10_cnn_solution.ipynb

# number of epochs to train the model
n_epochs = 5

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_dataloader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_dataloader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(train_dataloader.sampler)
    valid_loss = valid_loss/len(valid_dataloader.sampler)
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

--->
--->
--->


KeyboardInterrupt: 

In [0]:
model.load_state_dict(torch.load('model_cifar.pt'))

In [0]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

# Load Test Data
test_dataloader = DataLoader(
    NewDatset(dataframe=test_df), batch_size=batch_size, shuffle=True, num_workers=0)

# Get the model and evaluate
model.eval()

for data, target in test_dataloader:
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())

    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        

# average test loss
test_loss = test_loss/len(test_dataloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

classes = ['angry', 'surprise', 'fearful', 'neutral', 'calm', 'happy', 'sad', 'disgust']

for i in range(len(classes)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))